In [0]:
#Uncomment the following line of code to install ipywidgets
#!pip install ipywidgets  # installing widgets required for interactive sliders

import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import random
from math import pi,cos,sin,sqrt, log

from scipy.stats.distributions import chi2   #chi2.ppf(0.975, df=2)


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.special import expit

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [0]:
def Cov_matrix(L,theta=pi/4):
  '''
  The function returns a 2x2 covariance matrix parmeterized by L and theta
  The covariance matrix is generated by R*[[L,0],[0,1]]*R.T
  where R is the rotation matrix


  '''

  Rot_matrix=np.array([[cos(theta),-sin(theta)],[sin(theta),cos(theta)]])
 
  M=np.array([[L,0],[0,1]])
 # print("Dot product",np.dot(Rot_matrix,M))
  cov=np.dot(np.dot(Rot_matrix,M),Rot_matrix.T)
  

  return cov



def plot_ellipse(m,S):
  '''
  Plot an Elliptical contour of a bivariate Gaussian density

  Input:
  -m:mean
  -S:covariance matrix

  Output:
  -z : ellipse boundary points for plotting using
      >> plot(z[0,:],z[1,:])
  '''

  d=len(m)
  if d!=2:
    sys.exit("Plot ellipse assumes 2D data")
  
  #beta= theoretical mass enclosed
  beta=.9
  r=sqrt(chi2.ppf(beta, df=2))


  #Plot circle according to the given covariance matrix  -> https://cookierobotics.com/007/
  N=500
  t=np.linspace(0,2*pi,N+1)[:-1]
  u=r*np.array([np.cos(t),np.sin(t)])
  eigVal, eigVec = np.linalg.eig(S)

  z=np.dot(np.dot(eigVec,np.sqrt(np.diag(eigVal))),u) + m.reshape(-1,1)

  return z


def Mahalanobis_dist(x,mu,cov):
  '''
  Calculates Mahalanobis Distance
  -x: a 2D point ie of shape(2,)
  -mu: mean of shape(2,)
  -cov: Covariance matrix of shape(2,2)

  '''
  
  Cov_inv=np.linalg.inv(cov)
  x_mu=x-mu
  m_d=np.dot(x_mu.T, np.dot(Cov_inv,x_mu))


  return m_d

  




In [0]:
def LDA(X,y):
  '''
  -X : 2D data, of shape(N,2)
  -y : labels for the data, of shape(N,)
  '''
  # Calculating estimated parameters using MLE:
  sample={}
  sample[0]=X[y==0,:]
  sample[1]=X[y==1,:]

  p,mu={},{}
  p[0]=len(sample[0])/len(y)
  p[1]=len(sample[1])/len(y)
  #print("Probabitlites")
  #print(p[0],p[1])

  mu[0]=np.mean(sample[0],axis=0)
  mu[1]=np.mean(sample[1],axis=0)
  #print("Mean")
  #print(mu[0],mu[1])

  #Pooled covariance estimate:
  term1=sample[0].T-mu[0].reshape(-1,1)
  term2=sample[1].T-mu[1].reshape(-1,1)
  print(term1.shape, term2.shape)
  x_minus_mu=np.concatenate((term1,term2),axis=1)

  Cov=(1/len(X))*np.dot(x_minus_mu,x_minus_mu.T)

  
  # Plot Decision boundary 
 
 
  # x1 = np.linspace(-6,6,100)
  # x2 = np.linspace(-6,6,100)
  # xx1,xx2 = np.meshgrid(x1,x2)
  # xx = np.zeros((x1.shape[0]*x2.shape[0],2))
  # print(len(xx))
  # xx[:,0] = xx1.ravel()
  # xx[:,1] = xx2.ravel()
  # Z=[]

  # for idx in range(len(xx)):
  #   C0=log(p[0])-(1/2)*Mahalanobis_dist(xx[idx,:],mu=mu[0],cov=Cov)
  #   C1=log(p[1])-(1/2)*Mahalanobis_dist(xx[idx,:],mu=mu[1],cov=Cov)
  #   diff=C0-C1
  #   if diff>0:
  #    Z.append(-1)
  #   else:
  #    Z.append(1)

  # Z=np.asarray(Z)
  # Z=Z.reshape(-1,1)


  # #plt.contourf(x1,x2,Z.reshape((x1.shape[0],x2.shape[0])),alpha=0.3,level=[0],colours=['purple','red'],linestyles='solid',linewidths=1)
  # plt.contour(x1,x2,Z.reshape((x1.shape[0],x2.shape[0])),alpha=0.8,linestyles='solid',linewidths=2,colors=['black'],levels=['0.1'])
  y_intercept=log(p[0])/log(p[1])-(1/2)*np.dot(mu[0].T,mu[0])+(1/2)*np.dot(mu[1].T,mu[1])
  temp=np.dot(np.linalg.inv(Cov),mu[0]-mu[1])
  print(temp.shape)
  Slope=-temp[0]/temp[1]
  y_intercept=-y_intercept/temp[1]
  x_vec=np.linspace(-15,15)
  
  plt.plot(x_vec,Slope*x_vec/y_intercept)
  plt.title('LDA')



In [0]:
def LDA_predefined(X,y):
    lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
    y_pred = lda.fit(X, y).predict(X)

      
    x1 = np.linspace(-15,15,200)
    x2 = np.linspace(-15,15,200)
    xx1,xx2 = np.meshgrid(x1,x2)
    xx = np.zeros((x1.shape[0]*x2.shape[0],2))
    xx[:,0] = xx1.ravel()
    xx[:,1] = xx2.ravel()
    Z=[]
  
 
    Z = lda.predict_proba(xx)

    #print(Z)
    #Z = Z[:, 1].reshape(xx1.shape)
    Z = Z[:, 0].reshape(-1,1)
    idx=range(len(xx))
    Z[Z==0] = -1
   

    

    plt.contour(x1,x2,Z.reshape((x1.shape[0],x2.shape[0])),alpha=0.8,linestyles='solid',linewidths=2,colors=['black'],levels=['0.5'])
    plt.title('LDA')



In [0]:

def plot_MVG(parameter=1,theta=0,LDA_type='predefined'):
  mu={}
  Sigma={}
  sample={}
  c={0:'purple',1:'yellow'}
  sym={0:'.',1:'x'}
  mu[0]=np.array([-5,0])
  Sigma[0]=Cov_matrix(parameter,theta)
 

  mu[1]=np.array([5,0])
  Sigma[1]=Sigma[0]



  #sample some points
  for k in range(len(mu)):
    sample[k]=np.random.multivariate_normal(mu[k],Sigma[k],300)
  
  
  
  plt.figure(figsize=(7,7))
  plt.axis('equal')
  plt.xlim(-15,15)
  plt.ylim(-15,15)
  

  for k in range(len(mu)):
    plt.plot(sample[k][:,0],sample[k][:,1],sym[k],color=c[k])
    z=plot_ellipse(mu[k],Sigma[k])
    plt.plot(z[0,:],z[1,:],color=c[k])


  # data to be passed to LDA function
  y0=np.zeros((len(sample[0]),))
  y1=np.ones((len(sample[0]),))
  X=np.concatenate((sample[0],sample[1]),axis=0)
  label=np.concatenate((y0,y1)) 
  
  if LDA_type=='predefined':
    LDA_predefined(X=X,y=label)
  else: 
    LDA(X=X,y=label)
    #pass
  

# The interactive plot

lambda_widget=widgets.IntSlider(value=1,
    min=1,
    max=25,
    step=1,
    description='Lambda',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

theta_widget=widgets.FloatSlider(value=45*(pi/180),
    min=0,
    max=360*(pi/180),
    step=pi/180,
    description='theta',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    )

%time interact(plot_MVG,parameter=lambda_widget,theta=theta_widget,continuous_update=False)








interactive(children=(IntSlider(value=1, continuous_update=False, description='Lambda', max=25, min=1), FloatS…

CPU times: user 244 ms, sys: 112 ms, total: 355 ms
Wall time: 299 ms


<function __main__.plot_MVG>

## ALTERNATIVE: 
Wihout using predefined python libraries for LDA

In [0]:
# The interactive plot

lambda_widget=widgets.IntSlider(value=1,
    min=1,
    max=25,
    step=1,
    description='Lambda',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

theta_widget=widgets.FloatSlider(value=45*(pi/180),
    min=0,
    max=360*(pi/180),
    step=pi/180,
    description='theta',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    )

%time interact(plot_MVG,parameter=lambda_widget,theta=theta_widget,LDA_type='not predefined',continuous_update=False)

interactive(children=(IntSlider(value=1, continuous_update=False, description='Lambda', max=25, min=1), FloatS…

CPU times: user 239 ms, sys: 63.1 ms, total: 302 ms
Wall time: 252 ms


<function __main__.plot_MVG>